In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
from tqdm import tqdm
import seaborn as sns
%matplotlib inline
import train_util
import util
%load_ext autoreload
%autoreload 2
import keras
from keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras.optimizers import SGD

from keras.models import load_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
X, X_feat, Y, Y_no_sult = train_util.get_data(size=(128, 128, 3))

100%|██████████| 3887/3887 [00:31<00:00, 122.57it/s]


In [19]:
input_tensor = Input(shape=(128, 128, 3))
# dense_model = keras.applications.densenet.DenseNet121(include_top=False, weights=None, input_tensor=input_tensor, pooling=None)
xception_model = keras.applications.Xception(weights='imagenet',include_top=False, input_tensor=input_tensor)

In [54]:
# add FC layer
x = xception_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)
# top_model = Sequential()
# top_model.add(Flatten(input_shape=dense_model.output_shape[1:]))
# top_model.add(Dense(256, activation='relu', input_dim=feat_shape[1]* feat_shape[2]*feat_shape[3]))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(128, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(2, activation='softmax'))
# top_model.add(Dense(1, activation='sigmoid'))
model = Model(xception_model.input, predictions)


In [55]:
from sklearn.cross_validation import train_test_split
x_train, x_test, x_feat_train, x_feat_test, y_train, y_test = train_test_split(X, X_feat, Y_no_sult, test_size=0.1, random_state=43)
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

In [56]:
from keras.preprocessing.image import ImageDataGenerator
data_gen_args = {
                             'vertical_flip': True,
                             'horizontal_flip': True
}
# apply same args for generator
image_datagen = ImageDataGenerator(**data_gen_args)

# set same seed number and apply it to X and Y (mask)
seed = 1
image_datagen.fit(x_train, augment=True, seed=seed)

train_gen = image_datagen.flow(x_train, y_train, seed=seed, batch_size=32, shuffle=True)

# train_gen = zip(image_generator, feature_generator,  mask_generator)
val_gen = (x_test, y_test)

In [50]:
# train_features = xception_model.predict(x_train)
# feat_shape = np.array(train_features.shape)
# validation_features = xception_model.predict(x_test)

# train_features = np.reshape(train_features, (feat_shape[0], feat_shape[1]*feat_shape[2]*feat_shape[3]))
# validation_features = np.reshape(validation_features, (validation_features.shape[0], feat_shape[1]*feat_shape[2]*feat_shape[3]))


In [ ]:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = keras.optimizers.RMSprop(lr=2e-5)
train_util.train(model, train_gen, val_gen, steps_per_epoch=32, loss='categorical_crossentropy', optimizer=rmsprop )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Epoch 1/100
 - 17s - loss: 0.5922 - acc: 0.7188 - val_loss: 0.4694 - val_acc: 0.8329
Finished epoch: 0
{'val_loss': 0.4694123709416328, 'val_acc': 0.8329048844719914, 'loss': 0.5922468062490225, 'acc': 0.71875}

Epoch 00001: val_loss improved from inf to 0.46941, saving model to ./log/2018_0728_1659/best_weights.hdf5
Epoch 2/100
 - 11s - loss: 0.4760 - acc: 0.8379 - val_loss: 0.3632 - val_acc: 0.8689
Finished epoch: 1
{'val_loss': 0.3632231020682269, 'val_acc': 0.8688946015424165, 'loss': 0.4760086443275213, 'acc': 0.837890625}

Epoch 00002: val_loss improved from 0.46941 to 0.36322, saving model to ./log/2018_0728_1659/best_weights.hdf5
Epoch 3/100
 - 11s - loss: 0.3986 - acc: 0.8571 - val_loss: 0.2992 - val_acc: 0.8869
Finished epoch: 2
{'val_loss': 0.2991936009540043, 'val_acc': 0.8868894601542416, 'loss': 0.39689101014070644, 'acc': 0.8582834332527038}

Epoch 00003: val_loss improved from 0.36322 to 0.29919, saving model to ./log/2018_0728_1659/best_weights.hdf5
Epoch 4/100
 - 10s 

Finished epoch: 27
{'val_loss': 0.1651882622726166, 'val_acc': 0.9331619538807318, 'loss': 0.11909433320397511, 'acc': 0.9501953125}

Epoch 00028: val_loss did not improve
Epoch 29/100
 - 11s - loss: 0.1247 - acc: 0.9492 - val_loss: 0.1619 - val_acc: 0.9254
Finished epoch: 28
{'val_loss': 0.161942832880584, 'val_acc': 0.925449871618521, 'loss': 0.12474859732901677, 'acc': 0.94921875}

Epoch 00029: val_loss did not improve
Epoch 30/100
 - 11s - loss: 0.1372 - acc: 0.9558 - val_loss: 0.1698 - val_acc: 0.9332
Finished epoch: 29
{'val_loss': 0.1697505453558385, 'val_acc': 0.9331619538807318, 'loss': 0.12602702981816558, 'acc': 0.9590818364463166}

Epoch 00030: val_loss did not improve
Epoch 31/100
 - 11s - loss: 0.1259 - acc: 0.9482 - val_loss: 0.1680 - val_acc: 0.9306
Finished epoch: 30
{'val_loss': 0.16797175727650562, 'val_acc': 0.9305912597933281, 'loss': 0.12589008337818086, 'acc': 0.9482421875}

Epoch 00031: val_loss did not improve
Epoch 32/100
 - 11s - loss: 0.1410 - acc: 0.9492 - 

 - 11s - loss: 0.0643 - acc: 0.9854 - val_loss: 0.2153 - val_acc: 0.9383
Finished epoch: 59
{'val_loss': 0.21526750703857, 'val_acc': 0.9383033420555389, 'loss': 0.06431768939364702, 'acc': 0.9853515625}

Epoch 00060: val_loss did not improve
Epoch 61/100
 - 11s - loss: 0.0518 - acc: 0.9814 - val_loss: 0.2004 - val_acc: 0.9434
Finished epoch: 60
{'val_loss': 0.20035805761967343, 'val_acc': 0.9434447302303461, 'loss': 0.05182859123306116, 'acc': 0.9814453125}

Epoch 00061: val_loss did not improve
Epoch 62/100
 - 11s - loss: 0.0767 - acc: 0.9665 - val_loss: 0.2002 - val_acc: 0.9409
Finished epoch: 61
{'val_loss': 0.20024022827724564, 'val_acc': 0.9408740361429425, 'loss': 0.06989167800206623, 'acc': 0.9700598803584923}

Epoch 00062: val_loss did not improve
Epoch 63/100
 - 11s - loss: 0.0479 - acc: 0.9824 - val_loss: 0.2100 - val_acc: 0.9357
Finished epoch: 62
{'val_loss': 0.2099827470157324, 'val_acc': 0.9357326479681354, 'loss': 0.04787115612998605, 'acc': 0.982421875}

Epoch 00063: v

In [38]:
y_test.shape

(389,)

In [8]:
loaded_model = load_model('./log/2018_0728_1233/best_weights.hdf5')

In [9]:
preds_test = loaded_model.predict(x_test, verbose=1)

389/389 [==============================] - 6s 16ms/step


In [10]:
preds_test

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.9500689e-17],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.000000

In [17]:
import gc
del  dense_model
del  model
gc.collect()

NameError: name 'dense_model' is not defined